# 07 PyTorch Experiment Tracking

Pembelajaran Mesin adalah seebuah hal yang sangat expiremental

Unttuk menemukan eksperimen mana yang layak untuk dikejar, disitulah **epxeriment tracking** diperlukan.


In [10]:
import torch
import torchvision
from torchvision import transforms
from torchinfo import summary
from going_modular import data_setup, engine

import matplotlib.pyplot as plt

print(torch.__version__)
print(torchvision.__version__)

2.6.0+cu118
0.21.0+cu118


### Helper Function


In [11]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

In [13]:
set_seeds()

In [12]:
# Siapkan device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Ambil data

Ingin mengambil pizza, steak, sushi

Kita akan menjalankan eksperimen FoodVision Mini dan melihat model mana yang mantap!


In [33]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source:str,
                  destination: str,
                  remove_source: bool = True) -> Path:
  """Mengunduh dataset yang di zip, dan mengunzip sesuai destinasi"""

  #Setup path ke data folder
  data_path = Path("data/")
  image_path = data_path / destination

  # Jika folder gambar tidak ada, buat
  if image_path.is_dir():
    print(f"{image_path} sudah ada")
  else:
    print(f"Membuat {image_path}")
    image_path.mkdir(parents=True, exist_ok=True)

    # Download data target
    target_file = Path(source).name
    with open(data_path / target_file, "wb") as f:
      response = requests.get(source)
      print(f"Downloading {target_file} from {source}...")
      f.write(response.content)

    # Unzip target file
    with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
      print(f"Unzipping {target_file} data...")
      zip_ref.extractall(image_path)

    if remove_source:
      os.remove(data_path/target_file)

  return image_path

In [35]:
image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

Membuat data\pizza_steak_sushi
Unzipping pizza_steak_sushi.zip data...


WindowsPath('data/pizza_steak_sushi')

## 2. Buat datasets dan Dataloaders


### 2.1 Buat DataLoaders dengan manual transforms

Tujuan dengan transform adalah memastikan data kustom kita di format dengan bentuk yang sesuai dengan model pretrained kita


In [37]:
# Buat direktori

train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [38]:
# Buat ImageNet normalisasi levels

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Buat pipeline transform manual
from torchvision import transforms
manual_transforms = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.ToTensor(),
  normalize
])
print(f"Membuat transforms secara manual: {manual_transforms}")

Membuat transforms secara manual: Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [39]:
# Buat DataLoaders

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=manual_transforms,
                                                                               batch_size=32)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x227a2267c10>,
 ['pizza', 'steak', 'sushi'])

### 2.2 Buat DataLoaders dengan transform otomatis

Prinsip yang sama diterapkan denggan transform otomatis, dimana kita ingin data kustom kita memliki format yang sama dengan yang dibutuhkan oleh pretrained moodel kita


In [42]:
# Buat direktori
train_dir = image_path / "train"
test_dir = image_path / "test"

# Siapkan pretrained weights
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# Ambil transfrom dari weights nya
automatic_transforms = weights.transforms()
print(f"Membuat transforms secara otomatis: {automatic_transforms}")

# Buat DataLoaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=automatic_transforms,
                                                                               batch_size=32)

train_dataloader, test_dataloader, class_names

Membuat transforms secara otomatis: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x2279add5b50>,
 ['pizza', 'steak', 'sushi'])